In [32]:
import os
import sys
sys.path.append('../')
import time
import datetime
import numpy as np
import pandas as pd

from utils.tools import saveJson
from logger import logger
from utils.misc_utils import RMSE, MAE, masked_mape_np
from baseline.SOTABaselineMICE import  imputeMethodMICE as MICE
from fancyimpute import IterativeImputer
from ycimpute.imputer import mice
from utils.handle_missingdata import gene_missingdata, gene_missingdata_taxa_bias, gene_missingdata_chara_bias, \
    gene_missingdata_block_bias
from utils.dataAnalysis import analysisMeanMain
from myMethod import imputeMethod

In [33]:
#验证鸢尾花数据上测试本方法与MICE的区别
file = r'E:/labCode/autoencoders4imputation/public_data/1_Iris.xlsx'
data = pd.read_excel(file, sheet_name="dataset")
dt = np.array(data.values)
data = dt.astype('float')
originData = data[:-1]
target = data[-1]
for _ in range(5):
    result = {}
    for missPattern in ['normal']:
        for missRate in [0.05, 0.1, 0.2, 0.3, 0.4, 0.5]:
            if missPattern == 'normal':
                missData = gene_missingdata(rate=missRate, data=originData)
            elif missPattern == 'taxa':
                missData = gene_missingdata_taxa_bias(rate=missRate, data=originData)
            elif missPattern == 'chara':
                missData = gene_missingdata_chara_bias(rate=missRate, data=originData)
            elif missPattern == 'block':
                missData = gene_missingdata_block_bias(rate=missRate, data=originData)
            else:
                raise Exception("缺失模式错误，请在'normal','taxa','chara','block'中选择对应模式")
            result, MICEImputedData = MICE(result, originData, missData, missRate, missPattern)
            for firstImputedMethod in ['mice']:
                # 修改后，保存第一次预插补结果
                if firstImputedMethod=='ii':
                    firstImputedData=IterativeImputer().fit_transform(missData)
                elif firstImputedMethod=='mice':
                    firstImputedData  = MICEImputedData
                for loss in ['MSELoss']:
                    for autoMethod in ['Autoencoder','ResAutoencoder','StockedAutoencoder','StockedResAutoencoder']:
                        start=time.time()
                        result,_=imputeMethod(result=result,firstImputedMethod=firstImputedMethod,
                                            firstImputedData=firstImputedData.copy(),
                                            loss=loss,autoMethod=autoMethod,
                                            originData=originData,missData=missData,
                                            missRate=missRate,missPattern=missPattern)
                        logger.info("改后{}-{}-{}训练耗时:{}".format(firstImputedMethod,loss,autoMethod,time.time() - start))
        saveJson(result, "{}_{}_{}_{}.json".format("allMethod", missPattern, file.split("/")[-1],
                                                       datetime.datetime.now().strftime('%Y%m%d-%H%M%S')))

2021-01-18 20:06:43.951 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.21802902221679688
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:06:44.297 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-Autoencoder训练耗时:0.3450303077697754
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the 

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:06:50.833 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedAutoencoder训练耗时:0.5589704513549805
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:06:51.401 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedResAutoencoder训练耗时:0.5665059089660645
2021-01-18 20:06:51.689 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.2875511646270752
E:\labCode\autoencoders4imputation\basel

E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:06:58.425 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-ResAutoencoder训练耗时:0.34499669075012207
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:06

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:06.212 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedResAutoencoder训练耗时:0.6185071468353271
2021-01-18 20:07:06.525 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.31200218200683594
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:06.874 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-Autoencoder训练耗时:0.3474159240722656
E:\labCode\autoencoders4imputation\baseline\my

E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:14.466 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedAutoencoder训练耗时:0.616973876953125
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:22.275 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-Autoencoder训练耗时:0.35500049591064453
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:22.632 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-ResAutoencoder训练耗时:0.35610461235046387
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future wi

E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:29.372 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedResAutoencoder训练耗时:0.26399946212768555
2021-01-18 20:07:29.676 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.30300140380859375
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, 

..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:36.499 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-ResAutoencoder训练耗时:0.3552415370941162
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:37.112 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-StockedAutoencoder训练耗时:0.6127054691314697
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the futu

2021-01-18 20:07:44.515 | INFO     | utils.wapper:wapper:15 - imputeMethodMICE：耗时0.31754183769226074
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  elif self.firstImputedData!='None':
2021-01-18 20:07:44.876 | INFO     | __main__:<module>:37 - 改后mice-MSELoss-Autoencoder训练耗时:0.3600292205810547
E:\labCode\autoencoders4imputation\baseline\myMethod.py:27: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  isImputed= 'true' if firstImputedData!='None' else 'false'
..\dnn\tai_test.py:144: FutureWarning: elementwise comparison failed; returning scalar instead, but in the 

In [34]:
dataPath = r'E:\labCode\autoencoders4imputation\baseline'
savePath=r'C:\Users\pro\Desktop\自编码器鸢尾花结果验证'
analysisMeanMain(dataPath, savePath)

+------------------------------+--------+------------------------------------+----------+-------------+-------------+-------------+
|           dataSet            | Method |              pattern               | missRate |     RMSE    |     MAE     |     MAPE    |
+------------------------------+--------+------------------------------------+----------+-------------+-------------+-------------+
| allMethod_normal_1_Iris.xlsx | normal |      mice_MSELoss_Autoencoder      |   0.05   | 0.063+0.008 | 0.010+0.001 | 0.492+0.125 |
| allMethod_normal_1_Iris.xlsx | normal |    mice_MSELoss_ResAutoencoder     |   0.05   | 0.063+0.009 | 0.010+0.001 | 0.493+0.122 |
| allMethod_normal_1_Iris.xlsx | normal |  mice_MSELoss_StockedAutoencoder   |   0.05   | 0.064+0.009 | 0.010+0.002 | 0.506+0.116 |
| allMethod_normal_1_Iris.xlsx | normal | mice_MSELoss_StockedResAutoencoder |   0.05   | 0.063+0.009 | 0.010+0.001 | 0.489+0.123 |
| allMethod_normal_1_Iris.xlsx | normal |                MICE               